In [1]:
from datasets import load_dataset
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [15]:
ds = pd.read_csv('cust_support.csv')
texts = []
for i,j in zip(ds['Column2'], ds['Column5']):
    texts.append(f"q: {i} " + f"a: {j}")

In [29]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, convert_to_numpy=True)

dimension = embeddings.shape[1]
kb = faiss.IndexFlatL2(dimension)
kb.add(embeddings)
faiss.write_index(kb, "faiss.index")


In [17]:

def retrieve(query, top_k=2):
    query_emb = model.encode([query], convert_to_numpy=True)
    distances, indices = kb.search(query_emb, top_k)
    context = []
    category = ''
    for i, j in zip(indices[0], distances[0]):
        i = int(i)
        if i == -1:
            continue
        if j >= 0.3:
            context.append(f"q: {ds.iloc[i, 0]} " + f"a: {ds.iloc[i, 3]}")
            category = ds.iloc[i,1]
    return category, context

retrieve('Request for refund process clarification,Could you clarify the steps involved in requesting a refund? I submitted one last week but have no update.')

('REFUND',
 ["q: help me seeing your fucking money back policy a: I apologize for any inconvenience caused, but I'm unable to assist with that specific request. However, if you have any other questions or concerns about our refund policy, I'll be more than happy to provide you with the necessary information. Just let me know how I can assist you further.",
  'q: check your refund policy a: I\'ll make it happen! I understand the importance of having a clear understanding of our refund policy. Let me guide you on how you can access it.\r\n\r\nTo check our refund policy, you can visit our website and navigate to the "Terms and Conditions" or "Refund Policy" section. There, you will find comprehensive information about our refund process, including eligible situations, timeframes, and any specific requirements.\r\n\r\nIf you prefer, I can also provide you with a direct link to our refund policy page. Just let me know and I\'ll be more than happy to assist you further.\r\n\r\nPlease feel fr

In [26]:
from google import genai
from dotenv import load_dotenv
import os

def response(sender, query):
    load_dotenv()
    api_key = os.getenv('api_key')
    client = genai.Client(api_key=api_key)
    category, context = retrieve(query)
    prompt =  f"""Act as a professional email assistant. You will be given three inputs:
                The latest email or query that requires a response.
                The context, which includes the previous email responses or any relevant conversation history.
                Your goal is to generate a polite, clear, and contextually accurate reply that continues the conversation naturally. Keep the tone professional unless otherwise specified.

                Inputs:
                Team Name: Linkenite
                Work Time: 9AM - 5PM
                Sender: {sender}
                Query (incoming email): {query}
                Context (previous responses/known interactions): {context}

                Now, draft a well-structured email (body only) reply that is consistent with the context but directly addresses the new query. The response should:
                    - Acknowledge the sender’s concerns or questions by understanding their tone.
                    - Build on or reference relevant parts of the previous context.
                    - Provide clear next steps, answers, or required information and keep it short.
                    - Maintain an appropriate, professional closing.
                """


    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )

    return category, response.candidates[0].content.parts[0].text

In [27]:
category, auto_rep = response('Shreyas', 'General query about subscription,"Our servers are down, and we need immediate support. This is highly critical.')


In [28]:
print(category)
print("=====================")
print(auto_rep)

CONTACT
Dear Shreyas,

Thank you for reaching out to us. We understand the critical nature of your server issue and the urgency of needing immediate support.

For the fastest possible resolution to critical incidents like server downtime, we highly recommend contacting our dedicated support team directly. You can reach us by phone at **{{Customer Support Phone Number}}** or connect instantly with a representative through our Live Chat feature on our website at **{{Website URL}}**. Our team is available from **9 AM - 5 PM** to provide real-time assistance.

We are fully committed to resolving this for you as quickly as possible.

Sincerely,
The Linkenite Team


In [3]:
kb = faiss.read_index("faiss.index")
kb.ntotal

9979